In [ ]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
mdir = Path(Path(os.path.abspath('')).parent)
sys.path.append(str(mdir))
from copy import deepcopy
from copy import copy

import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv(mdir/'data'/'train.csv')

In [ ]:
test = pd.read_csv(mdir/'data'/'test.csv')


In [ ]:
train['is_train'] = 1
test['is_train'] = 0

y_train = train.per_square_meter_price.values

full_df = pd.concat([train, test]).reset_index(drop=True)

full_df.drop(['per_square_meter_price'], axis=1, inplace=True)

In [ ]:
train.date.min(), train.date.max()

In [ ]:
test.date.min(), test.date.max()

In [ ]:
train.price_type.unique(), test.price_type.unique()

In [ ]:
full_df.info()

In [ ]:
# predict final


from raif_hack.model import BenchmarkModel
from raif_hack.settings import MODEL_PARAMS, LOGGING_CONFIG, NUM_FEATURES, CATEGORICAL_OHE_FEATURES,CATEGORICAL_STE_FEATURES,TARGET

from raif_hack.utils import PriceTypeEnum
from raif_hack.metrics import metrics_stat
from raif_hack.features import prepare_categorical, get_time_features, change_target_inflation, get_territory_features
from raif_hack.features import get_random_feature, preproc_floors, fill_na, number_encode_features

MY_FEATURES = ['basement',
 'floor',
 'mezzanine',
 'num_floors',
 'osm_city_nearest_name',
 'per_square_meter_price',
 'price_type',
 'randNumCol',
 'street',
 'sum_build_001',
 'sum_build_001_diff',
 'sum_build_001_diff_regional',
 'sum_build_001_region',
 'sum_build_001_share',
 'sum_build_001_share_regional',
 'sum_build_005',
 'sum_build_005_diff',
 'sum_build_005_diff_regional',
 'sum_build_005_region',
 'sum_build_005_share',
 'sum_build_005_share_regional',
 'sum_build_0075',
 'sum_build_0075_diff',
 'sum_build_0075_diff_regional',
 'sum_build_0075_region',
 'sum_build_0075_share',
 'sum_build_0075_share_regional',
 'sum_build_01',
 'sum_build_01_diff',
 'sum_build_01_diff_regional',
 'sum_build_01_region',
 'sum_build_01_share',
 'sum_build_01_share_regional',
 'sum_other_001',
 'sum_other_001_diff',
 'sum_other_001_diff_regional',
 'sum_other_001_region',
 'sum_other_001_share',
 'sum_other_001_share_regional',
 'sum_other_005',
 'sum_other_005_diff',
 'sum_other_005_diff_regional',
 'sum_other_005_region',
 'sum_other_005_share',
 'sum_other_005_share_regional',
 'sum_other_0075',
 'sum_other_0075_diff',
 'sum_other_0075_diff_regional',
 'sum_other_0075_region',
 'sum_other_0075_share',
 'sum_other_0075_share_regional',
 'sum_other_01',
 'sum_other_01_diff',
 'sum_other_01_diff_regional',
 'sum_other_01_region',
 'sum_other_01_share',
 'sum_other_01_share_regional',
 'tech' ]

In [ ]:
"""train_df = pd.read_csv(mdir/'data'/'train.csv')
test_df = pd.read_csv(mdir/'data'/'test.csv')

train_df['is_train'] = 1
test_df['is_train'] = 0

full_df = pd.concat([train_df, test_df])"""


full_df = get_time_features(full_df)

full_df = get_territory_features(full_df)

# попробовать на честность
#full_df = change_target_inflation(full_df)

full_df = preproc_floors(full_df)
full_df = fill_na(full_df)

full_df = get_random_feature(full_df)

#full_df, _ = number_encode_features(full_df)

In [ ]:
full_df.sample(5)

In [ ]:
#full_df["per_square_meter_price"] = np.log1p(full_df["per_square_meter_price"])

In [ ]:
y_train_log = np.log1p(y_train)

In [ ]:
#full_df = 
#pd.get_dummies()
full_df[full_df.columns[full_df.dtypes == 'object']]

In [ ]:
import sklearn
from sklearn import preprocessing
import re

def number_encode_features(df):
    result = df.copy() 
    result['street'] += result['city']
    encoders = {}
    for column in result.columns[result.dtypes == 'object'].drop(['id']):
        encoders[column] = preprocessing.LabelEncoder() 
        result[column] = encoders[column].fit_transform(result[column])
    return result, encoders
full_df, _ = number_encode_features(full_df)

In [ ]:
model_cols = full_df.columns.drop(['id','date'])

In [ ]:
full_df_na = (full_df.isnull().sum() / len(full_df)) * 100
full_df_na = full_df_na.drop(full_df_na[full_df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :full_df_na})
missing_data.head(20)

In [ ]:
full_df.fillna(value = full_df['osm_city_nearest_population'].mean(), inplace=True)

In [ ]:
full_df_na = (full_df.isnull().sum() / len(full_df)) * 100
full_df_na = full_df_na.drop(full_df_na[full_df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :full_df_na})
missing_data.head(20)

In [ ]:
#train = full_df[:ntrain]
#test = full_df[ntrain:]
train_df = full_df[full_df['is_train'] == 1][model_cols]
test_df = full_df[full_df['is_train'] == 0][model_cols]

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_df.values)
    # y_train_log, y_train
    #rmse= np.sqrt(-cross_val_score(model, train_df.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    rmse= np.sqrt(-cross_val_score(model, train_df.values, y_train_log, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1, normalize=True, max_iter=1000000))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3, normalize=True, max_iter=1000000))
#KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5,)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_cat = CatBoostRegressor(iterations=30000,
                          loss_function = 'RMSE',
                          task_type="CPU",
                          depth=4  # - 81480.24422
                         )
grid = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300]}

#Instantiate GridSearchCV
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring ='accuracy', cv = 5)

#fit the model
gscv.fit(X,y)

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

In [ ]:

# Lasso score: 1.2285 (0.0390)
# Lasso score: 1.2287 (0.0407)   normalize=True,max_iter=1000000))  - 

score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
"""score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))"""

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_cat)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(train_df, y_train_log, test_size=0.1, random_state=42)

In [ ]:
model_cat = CatBoostRegressor(iterations=30000,
                          loss_function = 'MAPE',
                          task_type="CPU",
                          depth=4  # - 81480.24422
                         )


model_cat.fit(X = np.array(X_train),
            y = np.array(y_train),
          eval_set = (np.array(X_test), np.array(y_test)),
            #silent = False,
          verbose = 500,
            early_stopping_rounds=500, plot=True)

In [ ]:
train_df

In [ ]:
category_cols = ['wc','ed','ms','occ','rel','race','sex','nc','label']
for header in category_cols:
    train_set[header] = train_set[header].astype('category').cat.codes
    test_set[header] = test_set[header].astype('category').cat.codes

In [ ]:
RANDOM_SEED = 42
EARLY_STOPPING_ROUND = 500

def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 30000
    param['use_best_model'] = True
    param['eval_metric'] = 'RMSE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    #param['logging_level'] = 'Silent'
    param['verbose'] = 500
    
    regressor = CatBoostRegressor(**param)

    regressor.fit(X_train.copy(), y_train.copy(),
                  eval_set=[(X_test.copy(), y_test.copy())],
                  early_stopping_rounds=EARLY_STOPPING_ROUND)
    loss = mean_squared_error(y_test, regressor.predict(X_test.copy()))
    return loss
    

In [ ]:
import optuna
study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
study.optimize(objective, n_trials=30000, n_jobs=-1)

In [ ]:
study.best_value

In [ ]:
res = model_cat.predict(test_df)

sub = pd.DataFrame()
sub['id'] = test['id']
sub['per_square_meter_price'] = np.e**res*0.95
sub.to_csv('submission.csv',index=False)

In [ ]:
test['id']

In [ ]:
sub

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
    

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
sub = pd.DataFrame()
sub['id'] = test_ID
sub['SalePrice'] = np.e**res
sub.to_csv('submission.csv',index=False)

In [ ]:
train_df = full_df[full_df['is_train'] == 1]
test_df = full_df[full_df['is_train'] == 0]

In [ ]:
train_ID = train['id']
test_ID = test['id']

In [ ]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

#import pandas_profiling as pp # подсмотрел в kernels, интересно

import tensorflow as tf
from catboost import CatBoostRegressor, Pool

In [ ]:
import typing
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error

THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1
def deviation_metric_one_sample(y_true: typing.Union[float, int], y_pred: typing.Union[float, int]) -> float:
    """
    Реализация кастомной метрики для хакатона.

    :param y_true: float, реальная цена
    :param y_pred: float, предсказанная цена
    :return: float, значение метрики
    """
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    if np.abs(deviation) <= THRESHOLD:
        return 0
    elif deviation <= - 4 * THRESHOLD:
        return 9 * NEGATIVE_WEIGHT
    elif deviation < -THRESHOLD:
        return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
    elif deviation < 4 * THRESHOLD:
        return ((deviation / THRESHOLD) - 1) ** 2
    else:
        return 9



In [ ]:
true_features = ['lat',
 'lng',
 'osm_amenity_points_in_0.001',
 'osm_amenity_points_in_0.005',
 'osm_amenity_points_in_0.0075',
 'osm_building_points_in_0.005',
 'osm_building_points_in_0.0075',
 'osm_building_points_in_0.01',
 'osm_catering_points_in_0.001',
 'osm_catering_points_in_0.005',
 'osm_catering_points_in_0.0075',
 'osm_catering_points_in_0.01',
 'osm_city_closest_dist',
 'osm_city_nearest_population',
 'osm_crossing_closest_dist',
 'osm_crossing_points_in_0.001',
 'osm_crossing_points_in_0.005',
 'osm_crossing_points_in_0.0075',
 'osm_crossing_points_in_0.01',
 'osm_culture_points_in_0.005',
 'osm_culture_points_in_0.0075',
 'osm_culture_points_in_0.01',
 'osm_finance_points_in_0.001',
 'osm_finance_points_in_0.005',
 'osm_finance_points_in_0.0075',
 'osm_finance_points_in_0.01',
 'osm_healthcare_points_in_0.005',
 'osm_healthcare_points_in_0.0075',
 'osm_healthcare_points_in_0.01',
 'osm_historic_points_in_0.005',
 'osm_historic_points_in_0.0075',
 'osm_historic_points_in_0.01',
 'osm_hotels_points_in_0.0075',
 'osm_hotels_points_in_0.01',
 'osm_leisure_points_in_0.005',
 'osm_leisure_points_in_0.0075',
 'osm_leisure_points_in_0.01',
 'osm_offices_points_in_0.001',
 'osm_offices_points_in_0.005',
 'osm_offices_points_in_0.0075',
 'osm_offices_points_in_0.01',
 'osm_shops_points_in_0.001',
 'osm_shops_points_in_0.005',
 'osm_shops_points_in_0.0075',
 'osm_shops_points_in_0.01',
 'osm_subway_closest_dist',
 'osm_train_stop_closest_dist',
 'osm_train_stop_points_in_0.005',
 'osm_transport_stop_closest_dist',
 'osm_transport_stop_points_in_0.005',
 'osm_transport_stop_points_in_0.0075',
 'osm_transport_stop_points_in_0.01',
 'reform_count_of_houses_1000',
 'reform_count_of_houses_500',
 'reform_house_population_1000',
 'reform_house_population_500',
 'reform_mean_floor_count_1000',
 'reform_mean_floor_count_500',
 'reform_mean_year_building_1000',
 'reform_mean_year_building_500',
 'region',
 'total_square',
 'street',
 'realty_type',
 'day',
 'dayofyear',
 'sum_other_001',
 'sum_other_005',
 'sum_other_001_diff',
 'sum_other_005_diff',
 'sum_other_0075_diff',
 'sum_other_001_share',
 'sum_other_005_share',
 'sum_other_001_region',
 'sum_other_005_region',
 'sum_other_0075_region',
 'sum_other_01_region',
 'sum_other_001_diff_regional',
 'sum_other_005_diff_regional',
 'sum_other_0075_diff_regional',
 'sum_other_01_diff_regional',
 'sum_other_001_share_regional',
 'sum_other_005_share_regional',
 'sum_other_0075_share_regional',
 'sum_other_01_share_regional',
 'sum_build_005',
 'sum_build_005_share',
 'sum_build_005_region',
 'sum_build_0075_region',
 'sum_build_01_region',
 'sum_build_005_diff_regional',
 'sum_build_01_diff_regional',
 'sum_build_005_share_regional',
 'sum_build_01_share_regional']

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
X = train_df.drop(['per_square_meter_price','date', 'id', 'is_train'], axis=1)[true_features]
Y = train_df[['per_square_meter_price']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)



In [ ]:
from catboost import CatBoostRegressor, Pool


In [ ]:
model = CatBoostRegressor(iterations=30000,
                          loss_function = 'RMSE',
                          task_type="CPU",
                          depth=6  # - 81480.24422
                         )
                          #devices='0:1',
                          #learning_rate= 0.3,
                          #depth = 14,
                          #l2_leaf_reg = 1
                        

model.fit(X = np.array(X_train),
            y = np.array(y_train),
          eval_set = (np.array(X_test), np.array(y_test)),
            #silent = False,
          verbose = 500,
            early_stopping_rounds=20)

In [ ]:
'ok'

In [ ]:
mtest = test_df.drop(['per_square_meter_price','date'], axis=1)
mtest

In [ ]:
res = model_cat.predict(X_test)

In [ ]:
res

In [ ]:
import typing
THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error




def deviation_metric_one_sample(y_true: typing.Union[float, int], y_pred: typing.Union[float, int]) -> float:
    """
    Реализация кастомной метрики для хакатона.

    :param y_true: float, реальная цена
    :param y_pred: float, предсказанная цена
    :return: float, значение метрики
    """
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    if np.abs(deviation) <= THRESHOLD:
        return 0
    elif deviation <= - 4 * THRESHOLD:
        return 9 * NEGATIVE_WEIGHT
    elif deviation < -THRESHOLD:
        return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
    elif deviation < 4 * THRESHOLD:
        return ((deviation / THRESHOLD) - 1) ** 2
    else:
        return 9


def deviation_metric(y_true: np.array, y_pred: np.array) -> float:
    return np.array([deviation_metric_one_sample(y_true[n], y_pred[n]) for n in range(len(y_true))]).mean()

def median_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.median(np.abs(y_pred-y_true)/y_true)

def metrics_stat(y_true: np.array, y_pred: np.array) -> typing.Dict[str,float]:
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mdape = median_absolute_percentage_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    raif_metric = deviation_metric(y_true, y_pred)
    return {'mape':mape, 'mdape':mdape, 'rmse': rmse, 'r2': r2, 'raif_metric':raif_metric}

EPS = 1e-8
assert deviation_metric(np.array([1,2,3,4,5]),np.array([1,2,3,4,5])) <= EPS
assert deviation_metric(np.array([1,2,3,4,5]),np.array([0.9,1.8,2.7,3.6,4.5])) <= EPS
assert deviation_metric(np.array([1,2,3,4,5]),np.array([1.1,2.2,3.3,4.4,5.5])) <= EPS
assert deviation_metric(np.array([1,2,3,4,5]),np.array([1.15,2.3,3.45,4.6,5.75])) <= EPS
assert np.abs(deviation_metric(np.array([1,2,3,4,5]),np.array([1.3,2.6,3.9,5.2,6.5]))-1) <= EPS
assert np.abs(deviation_metric(np.array([1,2,3,4,5]),np.array([0.7,1.4,2.1,2.8,3.5]))-1*NEGATIVE_WEIGHT) <= EPS
assert np.abs(deviation_metric(np.array([1,2,3,4,5]),np.array([10,20,30,40,50]))-9) <= EPS
assert np.abs(deviation_metric(np.array([1,2,3,4,5]),np.array([0,0,0,0,0]))-9*NEGATIVE_WEIGHT) <= EPS
assert np.abs(deviation_metric(np.array([1,2,3,4,5]),np.array([1,2.2,3.3,5,50])) - 85/45) <= EPS

In [ ]:
"""{'mape': 1.5331790967384054,
 'mdape': 0.28967585280398345,
 'rmse': 74071.48571000711,
 'r2': 0.8060378887712242,
 'raif_metric': 3.3111947615740123}"""
metrics_stat(y_test, res)

In [ ]:
y_test.values.ravel()

In [ ]:
res = model.predict(np.array(test_df.drop(['per_square_meter_price','date', 'id', 'is_train'], axis=1)[true_features]))

In [ ]:
train_df[train_df['price_type'] == 1].per_square_meter_price.mean()


In [ ]:
res.mean()

In [ ]:
np.e**res

In [ ]:
res

In [ ]:
to_push = test_df[['id']]
to_push['per_square_meter_price'] = res

In [ ]:
test_sub = pd.read_csv(mdir/'data'/'test_submission.csv')


In [ ]:
test_sub = test_sub[['id']].merge(to_push, how='left', on='id')
test_sub

In [ ]:
test_sub.to_csv('res.csv', index=False)

In [ ]:
model.feature_importances_, model.feature_names_, test_df.drop(['per_square_meter_price','date', 'id', 'is_train'], axis=1).columns


In [ ]:
imp = []
feat = []
for x,y in zip(test_df.drop(['per_square_meter_price','date', 'id', 'is_train'], axis=1).columns, model.feature_importances_):
      if y > 0.2101:
            imp.append((x,y))
            feat.append(x)

In [ ]:
imp.sort(key=lambda x:x[1])

In [ ]:
feat

In [ ]:
to_test = pd.read_csv(mdir/'data'/'test.csv')
to_test

In [ ]:
test_df

In [ ]:
test_df.reset_index(drop=True, inplace=True)

In [ ]:
test_df['id'] = to_test['id']

In [ ]:
to_push = test_df[['id']]

In [ ]:
test_df[['id']]

In [ ]:
to_push

In [ ]:
to_push['per_square_meter_price'] = res

In [ ]:
to_push.to_csv('res.csv', index=False)